In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from skimage import io
from skimage import transform
from sklearn.utils import shuffle
from keras.utils import np_utils
import scipy.stats

In [2]:
frame = pd.read_csv('samples_with_marks.csv')
frame.head
filenames = np.asarray(frame['filename'])
result_ = np.asarray(frame['result'])

Y_ = np.ndarray((filenames.size, 32, 32))
for i in range(filenames.size):
    X = io.imread(filenames[i], as_gray = True) 
    #максимальная длина картинки в наших данных - 31
    X = np.r_[np.c_[X, np.ones((X.size // X[1].size, 32 - X[1].size))], np.ones((32-X.size // X[1].size, 32))] 
    Y_[i] = X
Y_ = 1-Y_
Y_ = Y_.astype('float32')

In [8]:
result_ = np.asarray([0 if result_[i] == 10 else 1 for i in range(len(result_))])

In [9]:
Y_, result_ = shuffle(Y_, result_)
Y_ = np.around(Y_)
Y_test = Y_[::5]
result_test = result_[::5]     
Y = np.delete(Y_, list(range(0, Y_.shape[0], 5)), axis=0)
result = np.delete(result_, list(range(0, result_.shape[0], 5)), axis=0)
num_classes = 2

In [11]:
model_211 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(input_shape=(32, 32, 1), filters=32, kernel_size=(5,5), padding='same', activation='relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='relu'),
    tf.keras.layers.Dropout(.65),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(.65),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256*8, activation=tf.nn.relu),
    tf.keras.layers.Dropout(.6),
    tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])
weights_file = 'weights.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(weights_file, monitor='acc')

model_211.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model_211.fit(Y.reshape(-1, 32, 32, 1), result, epochs=2, batch_size = 100, callbacks=[checkpoint])
model_211.evaluate(Y_test.reshape(-1, 32, 32, 1), result_test)

Epoch 1/2
47/47 [==============================] - 5s 96ms/step - loss: 0.0150 - accuracy: 0.9899
Epoch 2/2
37/37 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]